In [5]:
import os
import numpy as np
import matplotlib.pyplot as plt
import SimpleITK as sitk
import ipywidgets as widgets
from IPython.display import display

# 從我們自己的 utils 模組中匯入函式
from utils.data_loader import load_dicom_series
from utils.image_processing import register_images

# Matplotlib 設定，讓影像在 Notebook 中直接顯示
%matplotlib inline

載入病人

In [6]:
# --- 設定區 ---
# 您只需要修改這一行，可以輸入數字或字串
PATIENT_ID_INPUT = 1  # 您可以換成任何想測試的病人編號，例如 1, 15, "22"

# --- 自動格式化路徑 ---
# 將輸入的數字格式化為三位數的字串 (例如 1 -> "001", 22 -> "022")
PATIENT_ID_TO_TEST = f"{int(PATIENT_ID_INPUT):03d}"

# 資料路徑設定
BASE_DATA_PATH = "data/All_Patients"
T1C_FOLDER_NAME = "Ax T1 FS BH+C"
PDFF_FOLDER_NAME = "FatFrac  3D Ax IDEAL IQ BH"

# --- 載入資料 ---
print(f"正在準備載入病人 ID: '{PATIENT_ID_TO_TEST}' 的資料...")
t1c_dir = os.path.join(BASE_DATA_PATH, PATIENT_ID_TO_TEST, T1C_FOLDER_NAME)
pdff_dir = os.path.join(BASE_DATA_PATH, PATIENT_ID_TO_TEST, PDFF_FOLDER_NAME)

volume_t1c_sitk, array_t1c, _ = load_dicom_series(t1c_dir, f"T1+C ({PATIENT_ID_TO_TEST})")
volume_pdff_sitk, array_pdff, _ = load_dicom_series(pdff_dir, f"PDFF ({PATIENT_ID_TO_TEST})")

# 檢查是否成功載入
if array_t1c is not None and array_pdff is not None:
    print(f"\n單一病人 '{PATIENT_ID_TO_TEST}' 資料載入成功！")
else:
    print(f"\n資料載入失敗，請檢查路徑或病人 ID '{PATIENT_ID_TO_TEST}' 是否存在。")

正在準備載入病人 ID: '001' 的資料...
開始載入 T1+C (001) 從: data/All_Patients/001/Ax T1 FS BH+C
  T1+C (001) 載入成功。 NumPy 陣列形狀: (31, 512, 512)
開始載入 PDFF (001) 從: data/All_Patients/001/FatFrac  3D Ax IDEAL IQ BH
  PDFF (001) 載入成功。 NumPy 陣列形狀: (56, 256, 256)

單一病人 '001' 資料載入成功！


In [7]:
# --- 執行對位 ---
# 這會呼叫您在 utils/image_processing.py 中定義的函式
# 修改接收方式，以同時取得對位後的影像陣列與轉換物件
array_t1c_resampled, final_transform = register_images(volume_pdff_sitk, volume_t1c_sitk)

# --- 互動式視覺化與量化評估 ---
if array_t1c_resampled is not None:
    
    # --- 新增：計算 NDV (非微分同胚體積) ---
    print("\n--- 正在計算評估指標 ---")
    try:
        # 取得參考影像（固定影像）的資訊
        reference_image = volume_pdff_sitk
        
        # 將轉換變成一個位移場 (Displacement Field)
        displacement_field = sitk.TransformToDisplacementField(
            final_transform,
            sitk.sitkVectorFloat64,
            reference_image.GetSize(),
            reference_image.GetOrigin(),
            reference_image.GetSpacing(),
            reference_image.GetDirection()
        )
        
        # 計算位移場的雅可比行列式 (Jacobian Determinant)
        jacobian_det_image = sitk.DisplacementFieldJacobianDeterminant(displacement_field)
        jacobian_np = sitk.GetArrayFromImage(jacobian_det_image)
        
        # 計算 NDV (%)：計算行列式小於等於0的體素所佔的比例
        num_voxels = np.prod(jacobian_np.shape)
        num_non_diffeomorphic_voxels = np.sum(jacobian_np <= 0)
        ndv_percentage = (num_non_diffeomorphic_voxels / num_voxels) * 100.0
        
        print(f"✅ 非微分同胚體積 (NDV): {ndv_percentage:.6f} %")
        
    except Exception as e:
        print(f"❌ 計算 NDV 時發生錯誤: {e}")
    print("--------------------------\n")

    # --- 維持不變：互動式視覺化對位結果 ---
    def view_registered_slices(slice_idx):
        """
        一個根據傳入的 slice_idx 來繪製三張對比圖的函式。
        """
        fig, axes = plt.subplots(1, 3, figsize=(18, 6))

        # 影像 1: 原始 T1 (移動影像)
        # 由於原始 T1 的切片數可能不同，我們需要做邊界檢查
        if slice_idx < array_t1c.shape[0]:
            axes[0].imshow(array_t1c[slice_idx, :, :], cmap='gray')
            axes[0].set_title(f'Original T1c (Moving)\nSlice {slice_idx}')
        else:
            # 如果索引超出原始 T1 的範圍，就顯示一張黑色影像作為提示
            axes[0].imshow(np.zeros_like(array_pdff[0, :, :]), cmap='gray')
            axes[0].set_title(f'Original T1c (Out of Bounds)\nSlice {slice_idx}')
        axes[0].axis('off')

        # 影像 2: PDFF (目標影像)
        axes[1].imshow(array_pdff[slice_idx, :, :], cmap='gray')
        axes[1].set_title(f'PDFF (Fixed Target)\nSlice {slice_idx}')
        axes[1].axis('off')

        # 影像 3: 對位後的 T1
        axes[2].imshow(array_t1c_resampled[slice_idx, :, :], cmap='gray')
        axes[2].set_title(f'Registered T1c\nSlice {slice_idx}')
        axes[2].axis('off')

        plt.suptitle(f'Interactive Registration Viewer (Patient {PATIENT_ID_TO_TEST})', fontsize=16)
        plt.tight_layout()
        plt.show()

    # 建立互動式滑桿，並將它與上面的繪圖函式連結
    widgets.interact(
        view_registered_slices,
        slice_idx=widgets.IntSlider(
            min=0,
            max=array_pdff.shape[0] - 1, # 滑桿範圍以對位目標(PDFF)的總切片數為準
            step=1,
            value=array_pdff.shape[0] // 2, # 初始顯示中間的切片
            description='Slice Index:',
            continuous_update=False, # 拖動滑桿放開後才更新，效能較好
            layout=widgets.Layout(width='80%')
        )
    )
else:
    print("對位失敗或尚未執行，無法顯示互動式檢視器。")

--- 開始執行多階段影像對位 ---
  [階段 1/2] 正在進行仿射對位...
  仿射對位完成。最終度量值: -0.4033

  [階段 2/2] 正在進行 B-Spline 非剛性對位...
  B-Spline 對位完成。最終度量值: -0.4034

--- 對位完成，正在重採樣影像 ---

--- 正在計算評估指標 ---
✅ 非微分同胚體積 (NDV): 0.000000 %
--------------------------



interactive(children=(IntSlider(value=28, continuous_update=False, description='Slice Index:', layout=Layout(w…

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.filters import frangi, threshold_otsu
from skimage import exposure
import cv2
import ipywidgets as widgets

# ----- 步驟 1: 建立最終簡化版的互動式檢視器 -----
def interactive_final_viewer(slice_idx, vessel_scale):
    # 從全域變數獲取對位後的 T1 影像與原始的 PDFF 影像
    t1_slice = array_t1c_resampled[slice_idx, :, :]
    pdff_slice = array_pdff[slice_idx, :, :]
    
    # --- Frangi 血管偵測 ---
    vesselness = frangi(t1_slice, scale_range=(vessel_scale, vessel_scale + 4), scale_step=1, black_ridges=False)
    p2, p98 = np.percentile(vesselness, (2, 98))
    vesselness_rescaled = exposure.rescale_intensity(vesselness, in_range=(p2, p98))
    
    # --- 全自動血管遮罩閾值處理 (僅使用 Otsu) ---
    # 已移除手動調整功能
    if vesselness_rescaled.max() > 0:
        threshold_val = threshold_otsu(vesselness_rescaled[vesselness_rescaled > 0])
    else:
        threshold_val = 0.5 # 預設一個備用值
    vessel_mask = (vesselness_rescaled > threshold_val)

    # --- 繪製結果 (簡化為 1x2) ---
    # 已移除中間的 Frangi Vesselness 影像
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    
    axes[0].imshow(t1_slice, cmap='gray')
    axes[0].set_title(f'Registered T1c (Slice {slice_idx})')

    # --- 最終疊加影像 (疊在 PDFF 上) ---
    pdff_uint8 = exposure.rescale_intensity(pdff_slice, out_range=(0, 255)).astype(np.uint8)
    overlay_rgb = cv2.cvtColor(pdff_uint8, cv2.COLOR_GRAY2BGR)
    overlay_rgb[vessel_mask] = [255, 0, 0] # BGR for Blue

    axes[1].imshow(overlay_rgb)
    axes[1].set_title('Final Vessel Overlay on PDFF')

    for ax in axes:
        ax.axis('off')
    plt.tight_layout()
    plt.show()

# ----- 步驟 2: 啟動最終簡化版的互動式滑桿 -----
# 只保留 Slice Index 和 Vessel Scale 兩個調整項
widgets.interact(
    interactive_final_viewer,
    slice_idx = widgets.IntSlider(value=array_pdff.shape[0] // 2, min=0, max=array_pdff.shape[0] - 1, step=1, description='Slice Index:'),
    vessel_scale = widgets.IntSlider(value=1, min=1, max=10, step=1, description='Vessel Scale:')
)

interactive(children=(IntSlider(value=28, description='Slice Index:', max=55), IntSlider(value=1, description=…

<function __main__.interactive_final_viewer(slice_idx, vessel_scale)>